## Naive Bayes

In [ ]:
#Loading required packages
library(tidyverse)
library(ggplot2)
library(caret)
library(psych)
library(rpart)
library(randomForest)
library(Amelia)
library(mice)
library(e1071)
library(klaR)

### This example comes from [https://www.edureka.co/blog/naive-bayes-in-r/](https://www.edureka.co/blog/naive-bayes-in-r/)

### For a very nice manuall python tutorial see [https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/](https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/)

In [ ]:
diabetes = read.csv("diabetes.csv", header = T)
str(diabetes)

In [ ]:
describe(diabetes)

In [ ]:
#Convert '0' values into NA
diabetes[, 2:7][diabetes[, 2:7] == 0] <- NA

In [ ]:
#visualize the missing data
missmap(diabetes)

In [ ]:
###We can impute missing values using different techniques (k-mers, nearest neighbors)
mice_mod <- mice(diabetes[, c("Glucose","BloodPressure","SkinThickness","Insulin","BMI")], method='rf')
mice_complete <- complete(mice_mod)

In [ ]:
#Transfer the predicted missing values into the main diabetes set
diabetes$Glucose <- mice_complete$Glucose
diabetes$BloodPressure <- mice_complete$BloodPressure
diabetes$SkinThickness <- mice_complete$SkinThickness
diabetes$Insulin<- mice_complete$Insulin
diabetes$BMI <- mice_complete$BMI



In [ ]:
missmap(diabetes)

In [ ]:
#Data Visualization
ggplot(diabetes, aes(Age, colour = Outcome)) +
  geom_freqpoly(binwidth = 1) + 
  labs(title="Age Distribution by Outcome")

In [ ]:
#visual 2
c <- ggplot(diabetes, aes(x=Pregnancies, fill=Outcome, color=Outcome)) +
  geom_histogram(binwidth = 1) + labs(title="Pregnancy Distribution by Outcome")
c + theme_bw()

In [ ]:
#visual 3
P <- ggplot(diabetes, aes(x=BMI, fill=Outcome, color=Outcome)) +
  geom_histogram(binwidth = 1) + labs(title="BMI Distribution by Outcome")
P + theme_bw()

In [ ]:
#visual 4
ggplot(diabetes, aes(Glucose, colour = Outcome)) +
  geom_freqpoly(binwidth = 1) + labs(title="Glucose Distribution by Outcome")


In [ ]:
#visual 5
ggpairs(diabetes)

In [ ]:
#Building a model
#split data into training and test data sets
library(caret)
set.seed(998)
indxTrain <- createDataPartition(y = diabetes$Outcome,p = 0.75,list = FALSE)
training <- diabetes[indxTrain,]
testing <- diabetes[-indxTrain,]

In [ ]:
#Check dimensions of the split

prop.table(table(diabetes$Outcome)) * 100
prop.table(table(training$Outcome)) * 100
prop.table(table(testing$Outcome)) * 100

In [ ]:
#create objects x which holds the predictor variables and y 
#which holds the response variables
x = training[,-9]
y = training$Outcome

In [ ]:
model = train(x,y,'naive_bayes',trControl=trainControl(method='cv',number=10))
#?naive_bayes

In [ ]:
#Model Evaluation
#Predict testing set
Predict <- predict(model,newdata = testing )

In [ ]:
#Get the confusion matrix to see accuracy value and other parameter values

confusionMatrix(Predict, testing$Outcome )

In [ ]:
#Plot Variable performance
X <- varImp(model)
plot(X)